In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
dataset=pd.read_csv("Dataset.csv")

In [3]:
dataset.head()

,Day,Date,CodedDay,Zone,Weather,Temperature,Traffic
0,Wednesday,01-06-18,3,2,35,17,2
1,Wednesday,01-06-18,3,3,36,16,3
2,Wednesday,01-06-18,3,4,27,25,5
3,Wednesday,01-06-18,3,5,23,23,3
4,Wednesday,01-06-18,3,6,18,42,2


In [4]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder() 

In [5]:
dataset['Date']= le.fit_transform(dataset['Date'])

In [6]:
dataset.tail(10)

,Day,Date,CodedDay,Zone,Weather,Temperature,Traffic
1429,Friday,9,5,135,18,25,4
1430,Friday,9,5,136,31,8,2
1431,Friday,9,5,137,13,11,4
1432,Friday,9,5,138,34,15,1
1433,Friday,9,5,139,5,43,4
1434,Friday,9,5,140,47,38,1
1435,Friday,9,5,141,15,8,4
1436,Friday,9,5,142,26,38,1
1437,Friday,9,5,143,16,34,3
1438,Friday,9,5,144,16,17,1


In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1439 entries, 0 to 1438
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Day          1439 non-null   object
 1   Date         1439 non-null   int32 
 2   CodedDay     1439 non-null   int64 
 3   Zone         1439 non-null   int64 
 4   Weather      1439 non-null   int64 
 5   Temperature  1439 non-null   int64 
 6   Traffic      1439 non-null   int64 
dtypes: int32(1), int64(5), object(1)
memory usage: 73.2+ KB


In [8]:
X = dataset.iloc[:, 2:6].values
y = dataset.iloc[:, 6:7].values

In [9]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features (important for models like SVR, LinearRegression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:

# Dictionary to hold model performance
results = {}

# Define models
models = {
    'LinearRegression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'DecisionTree': DecisionTreeRegressor(),
    'RandomForest': RandomForestRegressor(),
    'XGBoost': XGBRegressor(),
    'SVR': SVR()
}

# Hyperparameter tuning using GridSearchCV (for example, for Random Forest and XGBoost)
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

param_grid_xgb = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7]
}

# Perform GridSearchCV for RandomForest
grid_search_rf = GridSearchCV(estimator=RandomForestRegressor(), param_grid=param_grid_rf, cv=3, n_jobs=-1, verbose=2)
grid_search_rf.fit(X_train_scaled, y_train)
best_rf = grid_search_rf.best_estimator_

# Perform GridSearchCV for XGBoost
grid_search_xgb = GridSearchCV(estimator=XGBRegressor(), param_grid=param_grid_xgb, cv=3, n_jobs=-1, verbose=2)
grid_search_xgb.fit(X_train_scaled, y_train)
best_xgb = grid_search_xgb.best_estimator_

# Evaluate models with cross-validation
for name, model in models.items():
    if name == 'RandomForest':
        model = best_rf
    elif name == 'XGBoost':
        model = best_xgb

    # Train the model
    model.fit(X_train_scaled, y_train)

    # Make predictions
    y_pred = model.predict(X_test_scaled)

    # Evaluate performance
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    accuracy = model.score(X_test_scaled, y_test)  # R^2 score

    # Store the results
    results[name] = {'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'Accuracy': accuracy}

# Show results
for name, metrics in results.items():
    print(f"{name} -> Accuracy: {metrics['Accuracy']*100:.2f}%, MAE: {metrics['MAE']:.2f}, MSE: {metrics['MSE']:.2f}, RMSE: {metrics['RMSE']:.2f}")

Fitting 3 folds for each of 36 candidates, totalling 108 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
LinearRegression -> Accuracy: -0.21%, MAE: 1.27, MSE: 2.13, RMSE: 1.46
Lasso -> Accuracy: -0.28%, MAE: 1.27, MSE: 2.13, RMSE: 1.46
Ridge -> Accuracy: -0.21%, MAE: 1.27, MSE: 2.13, RMSE: 1.46
DecisionTree -> Accuracy: -106.70%, MAE: 1.72, MSE: 4.40, RMSE: 2.10
RandomForest -> Accuracy: 0.16%, MAE: 1.28, MSE: 2.12, RMSE: 1.46
XGBoost -> Accuracy: 0.09%, MAE: 1.27, MSE: 2.12, RMSE: 1.46
SVR -> Accuracy: -2.72%, MAE: 1.29, MSE: 2.18, RMSE: 1.48


In [14]:
# Show summary of all models with MAE, MSE, and RMSE
print("\n📊 Model Comparison:")
for model_name, metrics in results.items():
    print(f"{model_name} - MAE: {round(metrics['MAE'], 2)} - MSE: {round(metrics['MSE'], 2)} - RMSE: {round(metrics['RMSE'], 2)}")

# Find the best model based on MAE (you can use other metrics like MSE or RMSE depending on your preference)
best_model_name = min(results, key=lambda model: results[model]['MAE'])
best_accuracy = 100 - results[best_model_name]['MAE']  # Optional: if you want to derive an accuracy-like measure
best_model = models[best_model_name] if best_model_name not in ['RandomForest', 'XGBoost'] else globals()[f'best_{best_model_name.lower()}']

print(f"\n✅ Best Model: {best_model_name} with MAE: {round(results[best_model_name]['MAE'], 2)}")

# Save best model
import pickle
with open("best_traffic_model.pkl", "wb") as f:
    pickle.dump(best_model, f)



📊 Model Comparison:
LinearRegression - MAE: 1.27 - MSE: 2.13 - RMSE: 1.46
Lasso - MAE: 1.27 - MSE: 2.13 - RMSE: 1.46
Ridge - MAE: 1.27 - MSE: 2.13 - RMSE: 1.46
DecisionTree - MAE: 1.72 - MSE: 4.4 - RMSE: 2.1
RandomForest - MAE: 1.28 - MSE: 2.12 - RMSE: 1.46
XGBoost - MAE: 1.27 - MSE: 2.12 - RMSE: 1.46
SVR - MAE: 1.29 - MSE: 2.18 - RMSE: 1.48

✅ Best Model: Lasso with MAE: 1.27
